In [1]:
import numpy as np
import pandas as pd
from copy import copy
from pathlib import Path
from itertools import chain
from tqdm.notebook import tqdm

import data_tools.graphs as gt
from hetnet_ml.extractor import MatrixFormattedGraph, piecewise_extraction

/home/rogertu/projects/MechRepoNet/mrn_venv/lib/python3.8/site-packages/data_tools/df_processing.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
nodes = pd.read_csv('../2_pipeline/13a_Model_Prep_Holdout_Set/out/nodes.csv', dtype=str)
edges = pd.read_csv('../2_pipeline/13a_Model_Prep_Holdout_Set/out/edges.csv', dtype=str)

mg = MatrixFormattedGraph(nodes, edges, 'ChemicalSubstance', 'Disease', max_length=4, w=0.4, n_jobs=30)

Processing node and edge data...
Initializing metagraph...
Generating adjacency matrices...


100%|███████████████████████████████████████████████████████████████████████████████████| 68/68 [00:45<00:00,  1.50it/s]



Determining degrees for each node and metaedge


100%|███████████████████████████████████████████████████████████████████████████████████| 68/68 [00:24<00:00,  2.72it/s]



Weighting matrices by degree with dampening factor 0.4...


100%|██████████████████████████████████████████████████████████████████████████████████| 68/68 [00:00<00:00, 111.70it/s]


## Strange behavior

For whatever reason, the imports below result in the above initizalizaiton of mg not finishing, so we've loaded mg first, then will continue the imports

In [3]:
from scipy.sparse import issparse, csc_matrix, csr_matrix

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, chi2, RFE, SelectFromModel

from xgboost import XGBClassifier

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# Data Transformer

We use an inverse hyperbolic sine transformation to transform the features.

The transormation is thus:

$$sinh^{-1} \left( \frac{X_{mp}}{\sigma_{mp}} \right)$$

where $X_{mp}$ is the column in the dwpc feature matrix $X$ coreesponding to metapath $mp$ and $\sigma_{mp}$ is the mean of the column.

In [4]:
class MeanScaledArcsinhTransformer(TransformerMixin):
        
    def fit(self, X, y=None):
        if issparse(X):
            self.initial_mean_ = X.tocoo().tocsc().mean(axis=0).A[0]
        else:
            self.initial_mean_ = X.mean(axis=0)

        # If input was DataFrame, Converts resultant series to ndarray
        try:
            self.initial_mean_ = self.initial_mean_.values
        except:
            pass
        
        # If inital mean == 0, likely all values were zero
        # this prevents issues later.
        self.initial_mean_[np.where(self.initial_mean_ == 0.0)] = 1
        
        return self

    def transform(self, X, y=None):
        if issparse(X):
            return np.arcsinh(X.tocoo().tocsc().multiply(self.initial_mean_**-1))
        return np.arcsinh(X / self.initial_mean_)


# Homegrown Feature Selector

Runs 6 analysis on the traning data to select features.

1. Correlation to the output
2. Chi_squared test
3. Recurssive Feature Elimantion on a Ridge Regressor
4. Embedded Feature Selection from a Lasso Regressor
5. Embedded Feature Selection from a Randomn Forest Classifier
6. Embedded Feature Selection from a Gradient Boosting Classifier

Each analysis will select `num_feats` best features. The selected features will then by chosen via a voting method with `min_selections` out of the 6 elements required to for a feature to be kept. 


We have also added an option for `always_keep`:  This allows for domain expertise to be factored into the feature selection process.  In our case, we know some metapaths are specifically mechanistic, so we want to include those wherever possible

In [5]:
def cor_selector(X, y, feature_names, num_feats):
    cor_list = []
    # calculate the correlation with y for each feature
    for i in range(X.shape[1]):
        if issparse(X):
            x = X[:, i].A.reshape(len(y))
        else:
            x = X[:, i]
        cor = np.corrcoef(x, y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = np.array(feature_names)[np.argsort(np.abs(cor_list))[-num_feats:].tolist()].tolist()    
    # feature selection? 0 for not select, 1 for select
    return [True if i in cor_feature else False for i in feature_names]

def chi2_selector(X, y, num_feats):
    this_selector = SelectKBest(chi2, k=num_feats)
    this_selector.fit(X, y)
    return this_selector.get_support()

def rfe_selector(X, y, num_feats, random_state=None):
    this_selector = RFE(estimator=LogisticRegression(C=.1, solver='liblinear', random_state=random_state), 
                        n_features_to_select=num_feats, step=.2, verbose=5)
    this_selector.fit(X, y)
    return this_selector.get_support()

def embeded_lr_selector(X, y, num_feats, random_state=None):
    this_selector = SelectFromModel(LogisticRegression(penalty="l1", solver='liblinear', random_state=random_state), 
                                    max_features=num_feats)
    this_selector.fit(X, y)

    return this_selector.get_support()

def embeded_rf_selector(X, y, num_feats, n_jobs, random_state=None):
    rfc = RandomForestClassifier(n_estimators=100, max_depth=50, n_jobs=n_jobs, random_state=random_state)
    this_selector = SelectFromModel(rfc, max_features=num_feats)
    this_selector.fit(X, y)
    return this_selector.get_support()

def embeded_xgb_selector(X, y, num_feats, n_jobs=1, random_state=None):
    # XGBoost takes 0 as default random state
    if random_state is None:
        random_state = 0
    # Paramaters optimized for speed, rather than accuracy (as we have 5 other estimators also providing votes)
    xgbc = XGBClassifier(max_depth=5, n_estimators=200, learning_rate=.16, min_child_weight=1, colsample_bytree=.8,
                         n_jobs=n_jobs, random_state=random_state)
    this_selector = SelectFromModel(xgbc, max_features=num_feats)
    this_selector.fit(X, y)
    return this_selector.get_support()


class FeatureSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, num_features=100, min_selections=4, n_jobs=1, feature_names=None, always_keep=None,
                 random_state=None):
        self.num_features = num_features
        self.min_selections = min_selections
        self.n_jobs = n_jobs
        self.feature_names = feature_names
        self.always_keep = always_keep
        self.random_state = random_state
        
    def fit(self, X, y=None):
                
        X_norm = MaxAbsScaler().fit_transform(X)
        if issparse(X):
            if type(X) != csc_matrix:
                X = X.tocsc()
            X_norm = X_norm.tocsc()
        
        print('Running Cor')
        cor_support = cor_selector(X, y, self.feature_names, self.num_features)
        print('Running Chi2')
        chi_support = chi2_selector(X_norm, y, self.num_features)
        print('Running RFE')
        rfe_support = rfe_selector(X_norm, y, self.num_features, self.random_state)
        print('Running LR')
        embeded_lr_support = embeded_lr_selector(X_norm, y, self.num_features, self.random_state)
        print('Running RF')
        embeded_rf_support = embeded_rf_selector(X, y, self.num_features, 
                                                 n_jobs=self.n_jobs, random_state=self.random_state)
        print('Running XG')
        embeded_xgb_support = embeded_xgb_selector(X, y, self.num_features, 
                                                   n_jobs=self.n_jobs, random_state=self.random_state)
        
        feature_selection_df = pd.DataFrame({'feature':self.feature_names, 'pearson':cor_support, 'chi_2':chi_support, 
                                             'rfe':rfe_support, 'logistics':embeded_lr_support,
                                             'random_forest':embeded_rf_support, 'xgboost':embeded_xgb_support})  

        feature_selection_df['total'] = np.sum(feature_selection_df, axis=1)
        self.feature_selection_df_ = feature_selection_df
        
        keep_features = feature_selection_df.query('total >= {}'.format(self.min_selections))['feature'].tolist()
        
        # Keep the features that we always want (e.g. domain expertise)
        if self.always_keep is not None:
            keep_features.extend(self.always_keep)
        
        self.keep_features_ = [f for f in self.feature_names if f in keep_features]
        
        return self
    
    def transform(self, X, y=None):
        
        if issparse(X) and type(X) != csc_matrix:
            X = X.tocsc()
        return X[:, [i for i, f in enumerate(self.feature_names) if f in self.keep_features_]]

In [6]:
def sparse_std(data, axis=1):                                                                              
    """take the standard deviation of a sparse matrix"""                  

    def get_vec_std(vec):                                                                                       
        return vec.A.std(ddof=1)                                                                                

    stds = []        
    
    # ensure the correct matrix type for easy row or column subsetting
    if axis==1 and type(data) != csc_matrix:
        data = data.tocoo().tocsc()
    if axis==0 and type(data) != csr_matrix:
        data = data.tocoo().tocsr()
    
    # Get the std for each vector along the given axis individually
    for i in range(data.shape[axis]):                                                                              
        if axis==1:
            stds.append(get_vec_std(data.getcol(i)))       
        elif axis==0:
            stds.append(get_vec_std(data.getrow(i)))       
        
    return np.array(stds)

In [7]:
def get_model_coefs(model, X, f_names):                                                                                 
    """Helper Function to quickly return the model coefs and correspoding fetaure names"""                              
                                                                                                                        
    # Ensure we have a numpy array for the features                                                                     
    if type(X) == pd.DataFrame:                                                                                         
        X = X.values
        
                                                                                                                        
    # Grab the coeffiencts                                                                                              
    coef = model.coef_                                                                                                  
    # Some models return a double dimension array, others only a single                                                 
    if len(coef) != len(f_names):                                                                                       
        coef = coef[0]                                                                                                  
                                                                                                                        
    # insert the intercept                                                                                              
    coef = np.insert(coef, 0, model.intercept_)                                                                         
    names = np.insert(f_names, 0, 'intercept')                                                                          
                                                                                                                        
    # Calculate z-score scaled coefficients based on the features                                                       
    if issparse(X):
        if type(X) != csc_matrix:
            X = X.tocoo().tocsc()
        z_intercept = coef[0] + sum(coef[1:] * X.mean(axis=0).A[0])
        z_coef = coef[1:] * sparse_std(X, axis=1)
        z_coef = np.insert(z_coef, 0, z_intercept)
    else:
        z_intercept = coef[0] + sum(coef[1:] * X.mean(axis=0))                                                              
        z_coef = coef[1:] * X.std(axis=0)                                                                                   
        z_coef = np.insert(z_coef, 0, z_intercept)                                                                          
                                                                                                                        
    # Return                                                                                                            
    return pd.DataFrame([names, coef, z_coef]).T.rename(columns={0:'feature', 1:'coef', 2:'zcoef'})    

In [8]:
float_size = 64 # bits
bits_per_gb = 8589934592

def print_mem_info(n_comp, n_dis, n_mps):
    print("{:,} Compounds * {:,} Diseases = {:,} C-D Pairs".format(n_comp, n_dis,
                                                                   n_comp * n_dis))
    print("{:,} C-D Pairs * {:,} Metapaths = {:,} Matrix Values".format(n_comp * n_dis,
                                                                    n_mps, 
                                                                    n_comp * n_dis * n_mps))

    print('{:1,.1f} GB of matrix values'.format(n_comp * n_dis * n_mps * float_size / (bits_per_gb)))
    
    print('{:1,.3f} GB per metapath'.format(n_comp * n_dis * float_size / (bits_per_gb)))

In [9]:
train_frac = 0.15
rs = 20200123


treat_comps = set(edges.query('type == "treats_CtD"')['start_id'])
# Sample the negatives and subsample 
keep_comps = set(nodes.query('id not in @treat_comps and label == "ChemicalSubstance"')
                      .sample(frac=train_frac*.10, random_state=rs)['id'])
# Then subsample the positives
keep_comps = keep_comps | set(nodes.query('id in @treat_comps')
                                   .sample(frac=train_frac, random_state=rs+1)['id'])

treat_dis = set(edges.query('type == "treats_CtD"')['end_id'])
# Sample the negatives and subsample cv
keep_dis = set(nodes.query('label == "Disease" and id not in @treat_dis')
                    .sample(frac=train_frac*.10, random_state=rs+2)['id'])
# Take the diseases Treated by these compounds
keep_dis = keep_dis | set(edges.query('type == "treats_CtD" and start_id in @keep_comps')['end_id'])

In [10]:
# MP counts will be the same with or without weights, so just use the original
all_mp_counts = pd.read_csv('../2_pipeline/13b_Model_Prep_Metapath_Membership_Analysis/out/all_mp_counts.csv')
all_mp_counts.head(2)

,mp,pair_count,subset,frac,sim_mp
0,CpoAawD,5854758,all_pairs,0.010287,False
1,CpoAsoD,0,all_pairs,0.000000,False


In [11]:
non_sim_names = all_mp_counts.query('sim_mp == False')['mp'].unique().tolist()
mp_qr = all_mp_counts.query('subset == "all_pairs" and mp in @non_sim_names and pair_count > 0')
good_mps = mp_qr['mp'].tolist()

In [12]:
print_mem_info(len(keep_comps), len(keep_dis), len(good_mps)) # 1 to 10

1,832 Compounds * 3,023 Diseases = 5,538,136 C-D Pairs
5,538,136 C-D Pairs * 7,012 Metapaths = 38,833,409,632 Matrix Values
289.3 GB of matrix values
0.041 GB per metapath


#### Original Memory info
`print_mem_info(len(keep_comps), len(keep_dis), len(good_mps))`
* 1,404 Compounds * 2,906 Diseases = 4,080,024 C-D Pairs
* 4,080,024 C-D Pairs * 7,012 Metapaths = 28,609,128,288 Matrix Values
* 213.2 GB of matrix values
* 0.030 GB per metapath


In [13]:
print('{:,} Positive training examples in subset'.format(len(edges.query('start_id in @keep_comps and type == "treats_CtD"'))))

8,607 Positive training examples in subset


#### Original Number of Positive training examples
`print('{:,} Positive training examples in subset'.format(len(edges.query('start_id in @keep_comps and type == "treats_CtD"'))))`

8,607 Positive training examples in subset


### Bring in the known metapaths found in DrugMechDB

In [14]:
mech_mps = pd.read_csv('../0_data/manual/mech_mps.txt', header=None)[0].values
dmdb_feat = set(all_mp_counts.query('mp in @mech_mps and subset == "all_pairs" and pair_count > 0 and sim_mp == False')['mp'])

len(dmdb_feat)

55

# Prepare output location

In [15]:
this_name = '13c_Model_Prep_Hyperparam_tuning_10'

out_dir = Path('../2_pipeline/test_ratio/').joinpath(this_name, 'out').resolve()
tmp_dir = out_dir.parent.joinpath('tmp')

out_dir.mkdir(parents=True, exist_ok=True)
tmp_dir.mkdir(parents=True, exist_ok=True)

# Extract the features one time

Use the pair counts to sort metapaths extaction as a niave load balancer

In [17]:
def sort_mps_for_pw_extraction(n_big_calcs, big_per_block, mp_list, frac_info):
    
    big_mp = frac_info.sort_values('frac', ascending=False).head(n_big_calcs)['mp'].tolist()
    other_mp = list(set(mp_list) - set(big_mp))

    block_size = len(other_mp) // (len(big_mp) // big_per_block)
    n_blocks = ((len(big_mp) + len(other_mp)) // block_size)

    out = []
    for i in range(n_blocks):
        for j in range(big_per_block):
            idx = i*big_per_block + j
            out.append(big_mp[idx])
        out += other_mp[i*block_size:(i+1)*block_size] 
        
    out += list(set(other_mp) - set(out))    
    
    return out, block_size

In [18]:
# 30, 5 finishes < 30 min. <- OVERFLOW
# 40, 8 finishes 18min 46s <- OVERFLOW
to_xtract, block_size = sort_mps_for_pw_extraction(100, 5, good_mps, mp_qr)

In [19]:
len(to_xtract), block_size

(7012, 345)

In [23]:
%%time
## Extract the metapaths to do some prep work
(pairs, feats), test_dwpc = piecewise_extraction(function=mg.extract_dwpc, 
                                 to_split='metapaths', block_size=block_size,
                                 axis=1,
                                 metapaths=to_xtract, 
                                 start_nodes=list(keep_comps), 
                                 end_nodes=list(keep_dis),
                                 return_sparse=True,
                                 sparse_df=False,
                                 n_jobs=30)

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [37:39<00:00, 107.58s/it]


CPU times: user 1min 57s, sys: 1min 35s, total: 3min 32s
Wall time: 37min 42s


# Training Targets for feature selection¶

In [24]:
pos_tups = edges.query('type == "treats_CtD"')[['start_id', 'end_id']].apply(tuple, axis=1).tolist()
pos_tups[:2]

[('CHEBI:100147', 'DOID:0050400'), ('CHEBI:100147', 'DOID:13148')]

In [25]:
pos_tups = set(pos_tups)

In [26]:
y = []
for row in tqdm(pairs.itertuples(), total=len(pairs)):
    if set([(row.chemicalsubstance_id, row.disease_id)]) & pos_tups:
        y.append(1)
    else:
        y.append(0)

y = np.array(y)
        
print(len(pairs), len(y), sum(y))

  0%|          | 0/5538136 [00:00<?, ?it/s]

5538136 5538136 8607


In [27]:
pairs['status'] = y

# Subsample the potential training examples

Our class is so imbalanced, to get a sizeable number of positive training examples, we end up with many orders of magnitude more negative examples.  Many of those examples will have no connections from the compound to the disease of interest, this providing a zero row in the matrix. We will not waste time training on those values, and instead focus on the ones that distinguish the positive from the negative training examples.

As this is just hyperparameter tunings, to speed things up, we will also limit the positive examples to a small portion of the negative examples, 10x (2 orders of magnitude) larger than the number of postitives. 

In [28]:
# Get the rows that are Non-zero
nz_index = pairs[test_dwpc.getnnz(1)>0].index

# have the number of postivies to get 100x this for the negatives.
n_pos = pairs['status'].sum()

# Sample the nonzero negative examples at a rate of 10x the positive samples
neg_index = pairs.loc[nz_index].query('status == 0').sample(n=10*n_pos, random_state=rs+10).sort_index().index

# and of course take the training postivies
pos_index = pairs.query('status == 1').index

# Union the two
train_index = pos_index.union(neg_index)

In [29]:
feats = np.array(feats)
nz_feats = feats[test_dwpc.getnnz(0)>0]
feat_index = test_dwpc.getnnz(0)>0

In [30]:
# Get our compounds as ndarrays for easy indexing with sklearns StratifiedKFold
keep_comps = np.array(list(keep_comps))
# Need to know how to properly stratify the split
is_treat_comp = np.array([1 if c in treat_comps else 0 for c in keep_comps])

## Select the features

This is a time consuming and costly step. We will do once with the initial DWPC for parameter tuning. We will perform again at the end with the selected parameters.

CPU times: user 35.2 s, sys: 6.47 s, total: 41.7 s
Wall time: 41.7 s

In [31]:
%%time
msat = MeanScaledArcsinhTransformer()
trans_dwpc = msat.fit_transform(test_dwpc[train_index][:, feat_index])

CPU times: user 4.37 s, sys: 336 ms, total: 4.71 s
Wall time: 4.71 s


In [32]:
%%time

if tmp_dir.joinpath('test_feats.txt').exists():
    keep_features = pd.read_csv(tmp_dir.joinpath('test_feats.txt'))['0'].tolist()
else:
    fsel = FeatureSelector(num_features=500, min_selections=4, n_jobs=30, 
                           feature_names=nz_feats.tolist(), always_keep=dmdb_feat, random_state=rs+5)
    sel_dwpc = fsel.fit_transform(trans_dwpc, y[train_index])
    keep_features = fsel.keep_features_
    pd.Series(keep_features).to_csv(tmp_dir.joinpath('test_feats.txt'), index=False)
    fsel.feature_selection_df_.to_csv(tmp_dir.joinpath('test_fs_df.csv'), index=False)

Running Cor


/home/rogertu/projects/MechRepoNet/mrn_venv/lib/python3.8/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/rogertu/projects/MechRepoNet/mrn_venv/lib/python3.8/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Running Chi2
Running RFE
Fitting estimator with 5937 features.
Fitting estimator with 4750 features.
Fitting estimator with 3563 features.
Fitting estimator with 2376 features.
Fitting estimator with 1189 features.
Running LR
Running RF
Running XG


/home/rogertu/projects/MechRepoNet/mrn_venv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:08:25] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/rogertu/projects/MechRepoNet/mrn_venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return reduction(axis=axis, out=out, **passkwargs)


CPU times: user 19min 30s, sys: 3.95 s, total: 19min 34s
Wall time: 2min 12s


## Prep for Hyperparam optimization

In [33]:
all_dwpc = dict()

def get_dwpc(w):
    global all_dwpc
    
    # only extract if not previously extracted
    if w not in all_dwpc.keys():
        mg.update_w(w)
        (pairs, feats), dwpc = mg.extract_dwpc(metapaths=keep_features, 
                                 start_nodes=list(keep_comps), 
                                 end_nodes=list(keep_dis),
                                 return_sparse=True,
                                 sparse_df=False,
                                 n_jobs=30)
        # next step is split, so need sparse rows
        dwpc = dwpc.tocoo().tocsr()
        #all_dwpc[w] = dwpc
        return dwpc
    else:
        return all_dwpc[w]

In [34]:
def neg_log_scal(val):
    """Scale values logrithmically. Larger input yields smaller result"""
    return (1-np.log1p(val)) 

def pos_log_scal(val):
    """Scale values logrithmically. Larger input yields larger result"""
    return np.log1p(val)


def auroc_mean_loss(auroc_mean, strength=2, scal_factor=.5):
    mean_roc_shift =  (np.abs(auroc_mean - 0.5)) / scal_factor
    baseline_roc = neg_log_scal(1)
    return (neg_log_scal(mean_roc_shift) - baseline_roc) * strength

def avg_pre_mean_loss(avg_pre_mean, strength=2.5, scal_factor=.15):
    avg_pre_mean_scal = avg_pre_mean / scal_factor
    baseline_prc = neg_log_scal(1)
    return (neg_log_scal(avg_pre_mean_scal) - baseline_prc) * strength

def auroc_std_loss(auroc_std, strength=2):
    return pos_log_scal(auroc_std) * strength

def avg_pre_std_loss(avg_pre_std, strength=2):
    return pos_log_scal(avg_pre_std) * strength

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def calc_loss(auroc_mean, avg_pre_mean, auroc_std, avg_pre_std):
    mean_roc_loss = auroc_mean_loss(auroc_mean)
    mean_pre_loss = avg_pre_mean_loss(avg_pre_mean)
    std_roc_loss = auroc_std_loss(auroc_std)
    std_pre_loss = avg_pre_std_loss(avg_pre_std)
    
    return sigmoid(mean_roc_loss + mean_pre_loss + std_roc_loss + std_pre_loss)-1
    
    

In [35]:
import pickle
from time import time

def hyperopt(param_space, y, num_eval):

    def objective_function(params):
        dwpc_params = {k.split('__')[1]: v for k, v in params.items() if k.split('__')[0] == 'dwpc'}
        enet_params = {k.split('__')[1]: v for k, v in params.items() if k.split('__')[0] == 'enet'}

        this_w = dwpc_params['w']

        # Set up the post feature extraction pipeline
        post_extraction_pipeline = Pipeline(
            [('transformer', MeanScaledArcsinhTransformer()),
             ('maxabs_scale', MaxAbsScaler()),
             ('e_net', LogisticRegression(penalty='elasticnet', solver='saga', max_iter=100, **enet_params,
                                          random_state=rs+6))], verbose=True)

        ## Get the dwpc information for the current pairs
        dwpc = get_dwpc(this_w)
        this_dwpc = dwpc[train_index]

        cv = cross_validate(post_extraction_pipeline, this_dwpc, y, cv=5, 
                            scoring=['average_precision', 'roc_auc'], return_estimator=True)

        # Write out scores for each run
        with open(tmp_dir.joinpath('scores_w_{0:1.4f}_C_{1:1.5f}_l1_{2:1.4f}.txt'.format(
                                   this_w, enet_params['C'], enet_params['l1_ratio'])), 'w') as f_out:

            f_out.write(', '.join([str(s) for s in cv['test_average_precision']]))
            f_out.write('\n')
            f_out.write(', '.join([str(s) for s in cv['test_roc_auc']]))
            f_out.write('\n')

        auroc_mean = cv['test_roc_auc'].mean()
        avg_pre_mean = cv['test_average_precision'].mean()
        auroc_std = cv['test_roc_auc'].std()
        avg_pre_std = cv['test_average_precision'].std()
        
        score = calc_loss(auroc_mean, avg_pre_mean, auroc_std, avg_pre_std)
        
        print('Mean AUROC: {:1.4f}'.format(auroc_mean))
        print('Mean Avg Pre: {:1.4f}'.format(avg_pre_mean))
        print('STD AUROC: {:1.4f}'.format(auroc_std))
        print('STD Avg Pre: {:1.4f}'.format(avg_pre_std))
        print('Loss: {:1.4f}'.format(score))
        
        return {'loss': score, 'status': STATUS_OK}

    start = time()
   
    
    trials = Trials()
    best_param = fmin(objective_function, 
                      param_space, 
                      algo=tpe.suggest, 
                      max_evals=num_eval, 
                      trials=trials,
                      rstate= np.random.RandomState(1))
    
    print(time() - start)
    return trials, best_param
    

In [36]:
# Previous best before adding new keep_features: 
# '1l_ratio': 0.10455936193818496, 'C': 0.000556880900960339, 'w': 0.2640929485381926
param_hyperopt = {
    'dwpc__w': hp.uniform('w', 0.01, 1),
    'enet__C': hp.loguniform('C', np.log(0.001), np.log(1)),
    'enet__l1_ratio': hp.uniform('l1_ratio', .01, .99),
}

In [37]:
trials, best_param = hyperopt(param_hyperopt, y[train_index], 50)

Changing w from 0.4 to 0.2471551211320533. Please wait...                                                               
  0%|                                                                            | 0/50 [00:00<?, ?trial/s, best loss=?]

  0%|          | 0/68 [00:00<?, ?it/s]

  1%|1         | 1/68 [00:00<00:26,  2.51it/s]

  9%|8         | 6/68 [00:00<00:04, 14.94it/s]

 19%|#9        | 13/68 [00:00<00:01, 28.04it/s]

 31%|###       | 21/68 [00:00<00:01, 41.04it/s]

 56%|#####5    | 38/68 [00:00<00:00, 72.87it/s]

 76%|#######6  | 52/68 [00:00<00:00, 85.74it/s]

 91%|#########1| 62/68 [00:01<00:00, 86.88it/s]

100%|##########| 68/68 [00:01<00:00, 58.93it/s]


Preparing function arguments...                                                                                         
Calculating DWPCs...                                                                                                    
  0%|                                                                            | 0/50 [00:06<?, ?trial/s, best loss=?]

  0%|          | 0/188 [00:00<?, ?it/s]

  1%|          | 1/188 [00:00<00:26,  6.99it/s]

  3%|2         | 5/188 [00:00<00:09, 19.09it/s]

  5%|4         | 9/188 [00:00<00:06, 26.76it/s]

  7%|6         | 13/188 [00:00<00:06, 29.11it/s]

  9%|9         | 17/188 [00:00<00:06, 26.95it/s]

 13%|#3        | 25/188 [00:00<00:04, 37.54it/s]

 15%|#5        | 29/188 [00:00<00:04, 35.88it/s]

 18%|#7        | 33/188 [00:01<00:04, 34.93it/s]

 20%|##        | 38/188 [00:01<00:03, 38.55it/s]

 24%|##4       | 46/188 [00:01<00:02, 49.46it/s]

 28%|##7       | 52/188 [00:01<00:02, 49.06it/s]

 31%|###       | 58/188 [00:01<00:02, 50.51it/s]

 34%|###4      | 64/188 [00:01<00:02, 48.86it/s]

 37%|###7      | 70/188 [00:01<00:02, 45.34it/s]

 40%|####      | 76/188 [00:01<00:02, 43.87it/s]

 43%|####3     | 81/188 [00:02<00:02, 42.36it/s]

 46%|####5     | 86/188 [00:02<00:02, 43.79it/s]

 49%|####9     | 93/188 [00:02<00:01, 48.58it/s]

 52%|#####2    | 98/188 [00:02<00:01, 45.01it/s]

 55%|#####4 

                                                                                                                        
Reshaping Result Matrices...
  0%|                                                                            | 0/50 [02:21<?, ?trial/s, best loss=?]

...

/home/rogertu/projects/MechRepoNet/mrn_venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "



[Pipeline] ....... (step 1 of 3) Processing transformer, total=   0.3s                                                  
[Pipeline] ...... (step 2 of 3) Processing maxabs_scale, total=   0.1s                                                  
[Pipeline] ............. (step 3 of 3) Processing e_net, total=   4.2s                                                  
 98%|████████████████████████████████████████████ | 49/50 [2:32:43<02:55, 175.47s/trial, best loss: -0.6244901564425244]

/home/rogertu/projects/MechRepoNet/mrn_venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "



[Pipeline] ....... (step 1 of 3) Processing transformer, total=   0.3s                                                  
[Pipeline] ...... (step 2 of 3) Processing maxabs_scale, total=   0.1s                                                  
[Pipeline] ............. (step 3 of 3) Processing e_net, total=   4.0s                                                  
Mean AUROC: 0.6956                                                                                                      
Mean Avg Pre: 0.3253                                                                                                    
STD AUROC: 0.1148                                                                                                       
STD Avg Pre: 0.0789                                                                                                     
Loss: -0.5137                                                                                                           
100%|███████████████████████████

/home/rogertu/projects/MechRepoNet/mrn_venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "



In [38]:
pickle.dump(best_param, open(out_dir.joinpath('best_param.pkl'), 'wb'))
pickle.dump(trials, open(out_dir.joinpath('trials.pkl'), 'wb'))

In [39]:
best_param

{'C': 0.13440967452889097,
 'l1_ratio': 0.22949839132699307,
 'w': 0.5477270574566522}

Previous `best_params`

{'C': 0.033729490904225834,
 'l1_ratio': 0.1653172093468481,
 'w': 0.5665616931869023}


# Use the selected params to do the feature selection


In [40]:
enet_params = {k: v for k, v in best_param.items() if k != 'w'}

post_extraction_pipeline = Pipeline(
    [('transformer', MeanScaledArcsinhTransformer()),
     ('feature_selection', FeatureSelector(num_features=500, min_selections=4, n_jobs=30,
                                           feature_names=to_xtract, always_keep=dmdb_feat,
                                           random_state=4))], verbose=True)

In [41]:
mg.update_w(best_param['w'])
(a, b), dwpc = piecewise_extraction(function=mg.extract_dwpc, 
                                 to_split='metapaths', block_size=block_size,
                                 axis=1,
                                 metapaths=to_xtract, 
                                 start_nodes=list(keep_comps), 
                                 end_nodes=list(keep_dis),
                                 return_sparse=True,
                                 sparse_df=False,
                                 n_jobs=30)

Changing w from 0.35287427504339774 to 0.5477270574566522. Please wait...


100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [37:05<00:00, 105.95s/it]


In [42]:
post_extraction_pipeline.fit(dwpc[train_index], y[train_index])

[Pipeline] ....... (step 1 of 2) Processing transformer, total=   2.8s
Running Cor


/home/rogertu/projects/MechRepoNet/mrn_venv/lib/python3.8/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/rogertu/projects/MechRepoNet/mrn_venv/lib/python3.8/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Running Chi2
Running RFE
Fitting estimator with 7012 features.
Fitting estimator with 5610 features.
Fitting estimator with 4208 features.
Fitting estimator with 2806 features.
Fitting estimator with 1404 features.
Running LR
Running RF
Running XG


/home/rogertu/projects/MechRepoNet/mrn_venv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:54:58] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[Pipeline] . (step 2 of 2) Processing feature_selection, total= 2.0min


/home/rogertu/projects/MechRepoNet/mrn_venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return reduction(axis=axis, out=out, **passkwargs)


Pipeline(steps=[('transformer',
                 <__main__.MeanScaledArcsinhTransformer object at 0x7f9199a16ee0>),
                ('feature_selection',
                 FeatureSelector(always_keep={'CaBPawD', 'CaGitThiBPawD',
                                              'CaGitTitBPawD', 'CaGnr>GnrBPawD',
                                              'CaGnr>GpoBPawD',
                                              'CaGnr>GprBPawD', 'CaGnr>GrxCmD',
                                              'CaGnrBPawD', 'CaGnrBPpoCmD',
                                              'CaGpoAcoBPawD', 'CaGpoApBPawD',
                                              'CaGpoAsoD', 'CaGpoBPawD',
                                              'CaGpoBPpoCmD'...
                                                'CaGnrBPcoAsoD', 'CinBPawPtGtD',
                                                'CrGnrBPpoCpvD',
                                                'CinBPnrGnrCpvD',
                                           

In [43]:
pe_feats = post_extraction_pipeline[1].keep_features_

In [44]:
len(pe_feats), len(keep_features) 

(205, 188)

In [45]:
len(set(keep_features) - set(pe_feats))

39

In [46]:
feat_df = post_extraction_pipeline[1].feature_selection_df_
feat_df.sort_values('total', ascending=False).to_csv(out_dir.joinpath('feature_selection_df.csv'), index=False)

In [47]:
pickle.dump(post_extraction_pipeline[1], open(out_dir.joinpath('feature_selector.pkl'), 'wb'))
pd.Series(post_extraction_pipeline[1].keep_features_).to_csv(out_dir.joinpath('kept_features.txt'), index=False)

In [48]:
enet_params = {k: v for k, v in best_param.items() if k != 'w'}


# Set up the post feature extraction pipeline
post_extraction_pipeline = Pipeline(
    [('transformer', MeanScaledArcsinhTransformer()),
     ('maxabs_scale', MaxAbsScaler()),
     ('e_net', LogisticRegression(penalty='elasticnet', solver='saga', max_iter=100, **enet_params,
                                  random_state=rs+6))], verbose=True)

## Get the dwpc information for the current pairs
(stuff), dwpc = mg.extract_dwpc(metapaths=pe_feats, 
                                start_nodes=list(keep_comps), 
                                end_nodes=list(keep_dis),
                                return_sparse=True,
                                sparse_df=False,
                                n_jobs=30)

Preparing function arguments...
Calculating DWPCs...


100%|█████████████████████████████████████████████████████████████████████████████████| 205/205 [02:17<00:00,  1.49it/s]



Reshaping Result Matrices...


100%|█████████████████████████████████████████████████████████████████████████████████| 205/205 [00:06<00:00, 32.94it/s]


Stacking columns...


In [49]:
this_dwpc = dwpc[train_index]
this_y = y[train_index]

In [50]:
cv = cross_validate(post_extraction_pipeline, this_dwpc, this_y, cv=5, 
                    scoring=['average_precision', 'roc_auc'], return_estimator=True)

auroc_mean = cv['test_roc_auc'].mean()
avg_pre_mean = cv['test_average_precision'].mean()
auroc_std = cv['test_roc_auc'].std()
avg_pre_std = cv['test_average_precision'].std()

score = calc_loss(auroc_mean, avg_pre_mean, auroc_std, avg_pre_std)

[Pipeline] ....... (step 1 of 3) Processing transformer, total=   0.3s
[Pipeline] ...... (step 2 of 3) Processing maxabs_scale, total=   0.1s
[Pipeline] ............. (step 3 of 3) Processing e_net, total=   7.3s
[Pipeline] ....... (step 1 of 3) Processing transformer, total=   0.3s
[Pipeline] ...... (step 2 of 3) Processing maxabs_scale, total=   0.1s
[Pipeline] ............. (step 3 of 3) Processing e_net, total=   8.5s
[Pipeline] ....... (step 1 of 3) Processing transformer, total=   0.3s
[Pipeline] ...... (step 2 of 3) Processing maxabs_scale, total=   0.1s
[Pipeline] ............. (step 3 of 3) Processing e_net, total=   7.4s
[Pipeline] ....... (step 1 of 3) Processing transformer, total=   0.3s
[Pipeline] ...... (step 2 of 3) Processing maxabs_scale, total=   0.1s
[Pipeline] ............. (step 3 of 3) Processing e_net, total=   8.0s
[Pipeline] ....... (step 1 of 3) Processing transformer, total=   0.3s
[Pipeline] ...... (step 2 of 3) Processing maxabs_scale, total=   0.1s
[Pipel

/home/rogertu/projects/MechRepoNet/mrn_venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [51]:
print(('AUROC Scores: '+5*'{:1.4f}, ').format(*cv['test_roc_auc'].tolist()))
print('Mean AUROC: {:1.4f}'.format(auroc_mean))
print(('Avg Pre. Scores: '+5*'{:1.4f}, ').format(*cv['test_average_precision'].tolist()))
print('Mean Avg Pre: {:1.4f}'.format(avg_pre_mean))
print('STD AUROC: {:1.4f}'.format(auroc_std))
print('STD Avg Pre: {:1.4f}'.format(avg_pre_std))
print('Loss: {:1.4f}'.format(score))

AUROC Scores: 0.7665, 0.7387, 0.7908, 0.7430, 0.6113, 
Mean AUROC: 0.7300
Avg Pre. Scores: 0.3626, 0.3194, 0.4616, 0.3930, 0.2747, 
Mean Avg Pre: 0.3623
STD AUROC: 0.0622
STD Avg Pre: 0.0637
Loss: -0.6140
